In [ ]:
!curl "https://raw.githubusercontent.com/sonlam1102/vihsd/refs/heads/main/data/vihsd.zip" --output vihsd.zip


In [ ]:
!unzip "./DePrBViHO/Dataset/ViHSD/vihsd.zip" -d "./DePrBViHO/Dataset/ViHSD"

# Load data and other stuff thing(do not care)

In [ ]:
import torch
import random
import numpy as np
import intel_extension_for_pytorch as ipex
import oneccl_bindings_for_pytorch

random.seed(1963)
np.random.seed(1963)
torch.manual_seed(1963)
device = torch.device('xpu' if torch.xpu.is_available() else 'cpu')


In [ ]:

import numpy as np
import pandas as pd

train = pd.read_csv('./DePrBViHO/Dataset/ViHSD/train.csv')
dev = pd.read_csv('./DePrBViHO/Dataset/ViHSD/dev.csv')
test = pd.read_csv('./DePrBViHO/Dataset/ViHSD/test.csv')


train['free_text'] = train['free_text'].astype(str)
X_train = train['free_text']
y_train = train["label_id"]

dev['free_text'] = dev['free_text'].astype(str)
X_valid = dev['free_text']
y_valid = dev['label_id']

test['free_text'] = test['free_text'].astype(str)
X_test = test['free_text']
y_test = test['label_id']



In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base-v2', clean_up_tokenization_spaces=False)

In [ ]:
import numpy as np
import re
import demoji
from underthesea import word_tokenize
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
from sklearn import preprocessing
import pickle
import os 

def preprocess(text):
    text = demoji.replace(text, "")
    text = word_tokenize(text, format="text")
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\s*([.,!?;])\s*', r'\1 ', text)
    text = re.sub(r'\s{2,}', ' ', text).strip()
    text = re.sub(r'\s([.,!?;])', r'\1', text)
    text = re.sub(r'([.,!?;])\s*', r'\1 ', text)
    text = text.lower()
    return text

def pre_process_features(X, batch_size=5000, index_file="processed_batches.txt", folder = "./Savedata_Train/"):
    os.makedirs(folder, exist_ok=True)
    num_batches = len(X) // batch_size + (1 if len(X) % batch_size != 0 else 0)
    
    # Mở file để ghi tên các batch
    with open(index_file, 'w') as index_f:
        with Pool(cpu_count()) as p:
            for i in tqdm(range(num_batches), desc="Processing Batches"):
                start_idx = i * batch_size
                end_idx = (i + 1) * batch_size
                batch_X = X[start_idx:end_idx]
                processed_batch_X = list(p.imap(preprocess, batch_X))

                # Lưu phần nhỏ đã xử lý vào tệp tạm thời
                batch_file = f"{folder}batch_{i}.pkl"
                with open(batch_file, 'wb') as f:
                    pickle.dump(processed_batch_X, f)
                # Ghi tên file vào index file
                index_f.write(f"{batch_file}\n")

            p.close()
            p.join()
def pre_process_features(X, batch_size=5000, index_file="processed_batches.txt", folder = "./Savedata/"):
    os.makedirs(folder, exist_ok=True)
    num_batches = len(X) // batch_size + (1 if len(X) % batch_size != 0 else 0)
    
    # Mở file để ghi tên các batch
    with open(folder + index_file, 'w') as index_f:
        with Pool(cpu_count()) as p:
            for i in tqdm(range(num_batches), desc="Processing Batches"):
                start_idx = i * batch_size
                end_idx = (i + 1) * batch_size
                batch_X = X[start_idx:end_idx]
                processed_batch_X = list(p.imap(preprocess, batch_X))

                # Lưu phần nhỏ đã xử lý vào tệp tạm thời
                batch_file = f"batch_{i}.pkl"
                with open(folder + batch_file, 'wb') as f:
                    pickle.dump(processed_batch_X, f)
                # Ghi tên file vào index file
                index_f.write(f"{batch_file}\n")

            p.close()
            p.join()
pre_process_features(X_train,  folder = "./Savedata_Train/")
pre_process_features(X_valid,  folder = "./Savedata_Vaild/")



In [ ]:
import numpy as np
import re
import pickle
from sklearn.model_selection import train_test_split

def load_processed_data(index_file="processed_batches.txt", folder = "./Savedata/"):
    processed_X = []
    with open(folder + index_file, 'r') as index_f:
        for batch_file in index_f:
            batch_file = folder + batch_file.strip()
            with open(batch_file, 'rb') as f:
                batch_X = pickle.load(f)
                processed_X.extend(batch_X)
            # Có thể xóa file sau khi tải nếu cần
            # os.remove(batch_file)
    return processed_X
X_train = load_processed_data(folder="./Savedata_Train/")
X_valid = load_processed_data(folder="./Savedata_Vaild/")


from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_train)
encoded_labels = le.transform(y_train)
encoded_test_labels = le.transform(y_valid)


In [ ]:
from tqdm import tqdm

def encoder_generator(sentences, labels):
    sent_index = []
    input_ids = []
    attention_masks = []

    for index, sent in tqdm(enumerate(sentences), total=len(sentences), desc="Encoding Sentences"):
        sent_index.append(index)

        encoded_dict = tokenizer.encode_plus(
            sent,
            add_special_tokens=True,
            max_length=20,
            pad_to_max_length=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)
    sent_index = torch.tensor(sent_index)

    return sent_index, input_ids, attention_masks, labels

train_sent_index, train_input_ids, train_attention_masks, train_encoded_label_tensors = encoder_generator(X_train, encoded_labels)
dev_sent_index, dev_input_ids, dev_attention_masks, dev_encoded_label_tensors = encoder_generator(X_valid, encoded_test_labels)

print('Original: ', X_train[0])
print('Token IDs:', train_input_ids[0])


In [ ]:
import pickle

# Save the training data to a file
with open('train_data.pkl', 'wb') as train_file:
    pickle.dump((train_sent_index, train_input_ids, train_attention_masks, train_encoded_label_tensors), train_file)

# Save the validation data to another file
with open('dev_data.pkl', 'wb') as dev_file:
    pickle.dump((dev_sent_index, dev_input_ids, dev_attention_masks, dev_encoded_label_tensors), dev_file)

print("Data saved successfully.")


In [ ]:
import pickle

# Load the training data
with open('train_data.pkl', 'rb') as train_file:
    train_sent_index, train_input_ids, train_attention_masks, train_encoded_label_tensors = pickle.load(train_file)

# Load the validation data
with open('dev_data.pkl', 'rb') as dev_file:
    dev_sent_index, dev_input_ids, dev_attention_masks, dev_encoded_label_tensors = pickle.load(dev_file)

print("Data loaded successfully.")


In [ ]:
# Connvert train, dev input by using TensorDataset

from torch.utils.data import TensorDataset,random_split

train_dataset = TensorDataset(train_input_ids,train_attention_masks,train_encoded_label_tensors)
dev_dataset = TensorDataset(dev_input_ids,dev_attention_masks,dev_encoded_label_tensors)

print('train data samples is {}'.format(len(train_dataset)))
print("valid data samples is {}".format(len(dev_dataset)))

In [ ]:

from torch.utils.data import DataLoader,RandomSampler,SequentialSampler

bs=128


train_data_loader = DataLoader(train_dataset,
                              sampler=RandomSampler(train_dataset),
                              batch_size=bs)
valid_data_loader = DataLoader(dev_dataset,
                              sampler=RandomSampler(dev_dataset),
                              batch_size=bs)

# Define model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, embedding_dim, n_filters, filter_sizes, output_dim, dropout, pad_idx):
        super().__init__()
        
        # Linear layer to process the input embedding
        self.fc_input = nn.Linear(embedding_dim, embedding_dim)
        
        # Convolutional layers with different filter sizes
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=embedding_dim,
                      out_channels=n_filters,
                      kernel_size=fs)
            for fs in filter_sizes
        ])
        
        # Fully connected layer to produce the final output
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        # Dropout for regularization
        self.dropout = nn.Dropout(dropout)

    def forward(self, encoded):
        """
        Forward pass of the CNN model.
        
        :param encoded: Tensor of shape [batch size, sentence length, embedding dimension]
        :return: Tensor of shape [batch size, output dimension]
        """
        
        # Apply the fully connected layer to the input
        embedded = self.fc_input(encoded)  # [batch size, sent len, emb dim]
        
        # Rearrange dimensions for Conv1d layer (expected input shape: [batch size, emb dim, sent len])
        embedded = embedded.permute(0, 2, 1)
        
        # Apply each convolutional layer and ReLU activation, then max-pool over the output
        pooled_outputs = []
        for conv in self.convs:
            conved = F.relu(conv(embedded))  # [batch size, n_filters, sent len - kernel_size + 1]
            pooled = F.max_pool1d(conved, conved.shape[2]).squeeze(2)  # [batch size, n_filters]
            pooled_outputs.append(pooled)
        
        # Concatenate all pooled outputs
        cat = self.dropout(torch.cat(pooled_outputs, dim=1))  # [batch size, n_filters * len(filter_sizes)]
        
        # Pass through the final fully connected layer to get output
        result = self.fc(cat)  # [batch size, output_dim]
        
        return result


In [ ]:
EMBEDDING_DIM = 768
N_FILTERS = 32
FILTER_SIZES = [1,2,3,5]
OUTPUT_DIM = len(le.classes_)
DROPOUT = 0.5
PAD_IDX = tokenizer.pad_token_id

cnn = CNN(EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)


In [ ]:
from transformers import AutoModel
phoBert = AutoModel.from_pretrained('vinai/phobert-base-v2')


In [ ]:
import torch.optim as optim

model_prameters = list(phoBert.parameters())+list(cnn.parameters())

optimizer = optim.AdamW(model_prameters,lr=2e-5,eps=1e-8, weight_decay=1e-5)

criterion = nn.CrossEntropyLoss()

In [ ]:
phoBert = phoBert.to(device)
cnn = cnn.to(device)
criterion = criterion.to(device)

cnn.train()
phoBert.train()
cnn, optimizer = ipex.optimize(cnn, optimizer=optimizer)
phoBert, optimizer = ipex.optimize(phoBert, optimizer=optimizer)


In [ ]:
# Calculate accuracy per batch during train

def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]]).xpu()

In [ ]:
# Def for training

from tqdm import tqdm

def train():

    epoch_loss = 0
    epoch_acc = 0

    phoBert.train()
    cnn.train()                                                                                                                                                                            

    for batch in tqdm(train_data_loader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        optimizer.zero_grad()

        embedded = phoBert(b_input_ids,b_input_mask)[0]

        predictions = cnn(embedded)

        loss = criterion(predictions, b_labels)

        acc = categorical_accuracy(predictions, b_labels)

        loss.requires_grad = True
        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(train_data_loader), epoch_acc / len(train_data_loader)

In [ ]:
# Class for predict label

import numpy as np

def predictions_labels(preds,labels):
    pred = np.argmax(preds,axis=1).flatten()
    label = labels.flatten()
    return pred,label

In [ ]:
# Evaluate loss, acc  and f1-macro

from sklearn.metrics import classification_report,accuracy_score,f1_score,recall_score

def eval():
    epoch_loss = 0
    
    total_predictions = []
    total_true = []

    all_true_labels = []
    all_pred_labels = []

    phoBert.eval()
    cnn.eval()
    
    with torch.no_grad():

        for batch in tqdm(valid_data_loader):
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            embedded = phoBert(b_input_ids,b_input_mask)[0]
            predictions = cnn(embedded)

            loss = criterion(predictions, b_labels)
            epoch_loss += loss.item()

            predictions = predictions.detach().cpu().numpy()

            label_ids = b_labels.to('cpu').numpy()

            pred,true = predictions_labels(predictions,label_ids)

            all_pred_labels.extend(pred)
            all_true_labels.extend(true)

    print(classification_report(all_pred_labels,all_true_labels))
    avg_val_accuracy = accuracy_score(all_pred_labels,all_true_labels)
    macro_f1_score = f1_score(all_pred_labels,all_true_labels,average='macro')

    avg_val_loss = epoch_loss/len(valid_data_loader)

    print("accuracy = {0:.2f}".format(avg_val_accuracy))

    return avg_val_loss,avg_val_accuracy,macro_f1_score

In [ ]:
# Time for training

import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
# Set device and gpu

n_gpu = torch.xpu.device_count()
torch.xpu.get_device_name(0)

phoBert.xpu()

# Training

In [ ]:

best_macro_f1 = float('0')
epochs = 50
for epoch in range(epochs):

    start_time = time.time()
    train_loss,train_acc = train()
    valid_loss,valid_acc,macro_f1 = eval()
    end_time = time.time()


    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if macro_f1 > best_macro_f1:
        best_macro_f1 = macro_f1
        torch.save(phoBert,'./phobert_cnn_model_part1_'+'task2a_2.pt')
        torch.save(cnn,'./phobert_cnn_model_part2_'+'task2a_2.pt')
        print("model saved")

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. acc: {valid_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. F1: {macro_f1*100:.2f}%')
    print('=============Epoch Ended==============')

In [ ]:
# Save PhoBERT and CNN

torch.save(phoBert,'module2_part1.pt')
torch.save(cnn,'module2_part2.pt')

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoConfig, AutoTokenizer, PreTrainedModel

# Tải PhoBERT và CNN đã lưu
phoBert = torch.load('module2_part1.pt')
cnn = torch.load('module2_part2.pt')


In [ ]:
class PhoBERTCNN(PreTrainedModel):
    config_class = AutoConfig

    def __init__(self, config, phoBert, cnn):
        super().__init__(config)
        self.phoBert = phoBert  # PhoBERT model
        self.cnn = cnn          # CNN model
    
    def forward(self, input_ids, attention_mask=None):
        # Get PhoBERT outputs
        phoBert_outputs = self.phoBert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = phoBert_outputs.last_hidden_state  # [batch size, seq len, hidden_dim]

        # Pass through the CNN model
        cnn_output = self.cnn(last_hidden_state)
        return cnn_output


In [ ]:
# Khởi tạo cấu hình PhoBERT
config = AutoConfig.from_pretrained('vinai/phobert-base-v2')
config.num_labels = 3  # Điều chỉnh theo số nhãn của bạn

# Khởi tạo mô hình kết hợp
combined_model = PhoBERTCNN(config, phoBert, cnn)

# Lưu mô hình và tokenizer
combined_model.save_pretrained("./PhoBERTCNN")
tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base-v2')
tokenizer.save_pretrained("./PhoBERTCNN")


In [ ]:
from transformers import AutoModel, AutoTokenizer

# Tải lại mô hình và tokenizer
loaded_model = CombinedPhoBERTCNN.from_pretrained("./PhoBERTCNN")
loaded_tokenizer = AutoTokenizer.from_pretrained("./PhoBERTCNN")
